In [6]:
import numpy as np 
import os
from os import listdir
from os.path import isfile, join
import open3d as o3d
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import cm


from utils import readData,readPointCloud


In [7]:
## Reading the Transformation Matrix
global_trans = readData("../../Data/dataset/01.txt") # Returns a 2D array with each row as the global poses at a 
                                            # given timestamp from the video camera's frame.

## Reading the PCDs
mypath='../../Data/dataset/01/'

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles=sorted(onlyfiles)

read_pc = []

for f in onlyfiles: # Interating through the 77 bin files 
    temp= readPointCloud(mypath+f)
    x = np.ones(temp.shape[0])
    temp[:,3] = x # Replacing the reflectance values with 1 as they are not needed.
    temp1=temp.T
    read_pc.append(temp1) # Storing all the point clouds in a list

In [8]:
## Lidar to  Camera Transformation
 
"""

The reasoning for the values in the transformation matrices below is given in the report.

"""
cam_trans = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]]) 
back_trans = np.array([[0,0,1],[-1,0,0],[0,-1,0]])

## Final set of points

final=[]

for i in range(77): #iterating through all the point clouds
    final.append((back_trans@(np.resize(global_trans[i],(3,4))@cam_trans@read_pc[i])).T) # Stroring the point clouds in a 
                                                                                     # after applying the required
                                                                                     # transformations
# print(final)
##----------- End------- ##

# Question 2

In [9]:
def two_two_one(final,threshold):
    for j in range(5):
        
        test = np.around(final[j]).astype(np.int16) # Converting the point cloud to integer values
        
        x_y = test[:,0:2] # Extracting the x and y coordinates
        
        # Getting the max and min values along the x and y coordinates
        max_x = np.amax(x_y[:,0])
        max_y = np.amax(x_y[:,1])
        min_x = np.amin(x_y[:,0])
        min_y = np.amin(x_y[:,1])

        len_x = max_x - min_x
        len_y = max_y - min_y
        
        #Creating the occupancy grid and initialising it with 0s
        occupancy_grid = np.zeros((abs(max_y-min_y)+1,abs(max_x-min_x)+1))
        
        # Eliminating any duplicates of points
        xy_u = np.unique(test,axis=0)
        
        # Mapping each point in the point cloud to a pixel in the occupancy grid
        for i in range(len(xy_u)):
            x = xy_u[i,0]
            y = xy_u[i,1]
            occupancy_grid[y-min_y,x-min_x]+=1
        
        # Removing the points that do not have enough z values
        occupancy_grid = (occupancy_grid>threshold)
        
        #Saving the image in the '../../Data/Results/2_2/1st/photos' folder
        index = str(j)
        path = '../../Data/Results/2_2/1st/photos/'
        image ="bin"+index+".png"
        path_final = path + image
        np.asarray(occupancy_grid)
        
        occupancy_grid = Image.fromarray(occupancy_grid)
        occupancy_grid.save(path_final, 'PNG')
        
#         plt.imshow(occupancy_grid,cmap='gray')
#         plt.show()
        
def two_two_two(final,count,threshold):  

    test = np.around(final[0]).astype(np.int16) # Converting the point cloud to integer values
    count = count-1
    
    # Mapping each point in the point cloud to a pixel in the occupancy grid
    # We only choose the first 5/10/15 point clouds as mentioned in the question.
    for j in range(count):
        test = np.append(test, np.around(final[j+1]).astype(np.int16), axis=0)
    
    count = count +1 
    
    # Getting the max and min values along the x and y coordinates
    max_x = np.amax(test[:,0])
    min_x = np.amin(test[:,0])
    max_y = np.amax(test[:,1])
    min_y = np.amin(test[:,1])
    
    #Creating the occupancy grid and initialising it with 0s
    occupancy_grid = np.zeros((abs(max_y-min_y)+1,abs(max_x-min_x)+1))
    
    # Eliminating any duplicates of points
    xy_u = np.unique(test,axis=0)
    
    # Mapping each point in the point cloud to a pixel in the occupancy grid
    for i in range(len(xy_u)):
        x = xy_u[i,0]
        y = xy_u[i,1]
        occupancy_grid[y-min_y,x-min_x]+=1
        
    # Removing the points that do not have enough z values
    occupancy_grid = (occupancy_grid>threshold)
    
    #Saving the image in the '../../Data/Results/2_2/2nd/' folder
    index = str(count)
    
    path = '../../Data/Results/2_2/2nd/'
    image ="bin"+index+".png"
    path_final = path + image
    np.asarray(occupancy_grid)

    occupancy_grid = Image.fromarray(occupancy_grid)
    occupancy_grid.save(path_final, 'PNG')
#     plt.imshow(occupancy_grid,cmap='gray')
#     plt.show()    


In [10]:
###2.2.1###
threshold1 = 1
two_two_one(final,threshold1)


## 2.2.2
threshold2 = 3
two_two_two(final,5,threshold2)
two_two_two(final,10,threshold2)
two_two_two(final,15,threshold2)